<a href="https://colab.research.google.com/github/rohitkumar4826/Finetuned_model/blob/main/Rag_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U \
    langchain \
    langchain-community \
    langchain-text-splitters \
    transformers \
    sentence-transformers \
    faiss-cpu \
    accelerate \
    pypdf \
    bitsandbytes

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# -------- Load PDF --------
loader = PyPDFLoader("PointCity_Rulebook_web.pdf")
documents = loader.load()

# -------- Split text --------
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
chunks = text_splitter.split_documents(documents)
print(f"Chunks created: {len(chunks)}")

# -------- Embeddings --------
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# -------- Vector Store (SAVE ONCE) --------
vectorstore = FAISS.from_documents(chunks, embeddings)
vectorstore.save_local("faiss_index")

print("Vector store saved.")

# -------- Load Small, Fast Model --------
model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# phi-2 has no PAD token → use EOS
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

print("Model & tokenizer loaded.")


In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)


In [ ]:
def ask_question(query):
    docs = vectorstore.similarity_search(query, k=3)

    # If nothing relevant retrieved
    if not docs:
        print("I don’t know based on the provided document.")
        return

    context = "\n".join(doc.page_content for doc in docs)

    prompt = f"""
You are a strict assistant.

Rules:
- Answer ONLY from the context
- If answer is not in context, say:
  "I don’t know based on the provided document."
- Be concise

Context:
{context}

Question:
{query}

Answer:
"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True
    ).to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=120,
            temperature=0.1,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    answer = decoded.split("Answer:")[-1].strip()

    print("Answer:")
    print(answer)
ask_question("Who is rule book")